In [1]:
# Import excel spreadsheet to pandas data frame
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import math

/Users/sean/.conda/envs/veeringReporting/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# Define path the folder and phase excel file from RRP
directory = '/Users/sean/Documents/FJ_round2/'  # job folder
file_phase='PhaseTableReport (1).xlsx' # phase report from RRP
headerRow = 1 # row number of first phase in phase report

In [5]:
# import using read_excel, using headercolumn as columnnames
phases_in = pd.read_excel(directory+file_phase, header=headerRow-1)
phases_in = phases_in.dropna(subset=['StartTime']) # remove excess rows


In [6]:
phases_in.columns

Index(['Type', 'Tack', 'StartTime', 'Duration', 'Sails', 'RaceLeg', 'Subject',
       'Comments', 'Density', 'TWS', 'TWA', 'Perf', 'Pol%', 'Vmg%', 'TWS_Bin',
       'TWA_Bin', 'BSP_Bin', 'Dist (Nm)', 'BSP', 'AHEEL', 'AWA', 'TRIM',
       'Base Rake', 'Chock', 'D1', 'D2', 'HDG', 'Headstay', 'HEEL', 'V1',
       'BSP_trg%', 'RUDDER', 'FORESTAY'],
      dtype='object')

In [8]:
# set categorical variable types
categoricalVariables = ['Sails','Tack', 'TWS_Bin', 'TWA_Bin','BSP_Bin', 'Headstay', 'V1', 'D1', 'D2', 'Base Rake', 'Chock']
dropped = []
for var in categoricalVariables: # iterate over all categorical variables
    phases_in[var] = phases_in[var].astype('category') # change type to catagorical
    catCount = phases_in.groupby(var).size() # find total number of each category
    catCount = catCount/sum(catCount) # calculate category size as fraction of all observations
    drop = catCount[catCount < 0.01].index # identify cases that represent less than 1%
    phases_in[var] = phases_in[var].cat.remove_categories(drop) # drop cases which represent less than 1%
    dropped.append(drop)
## Print dropped catagories
for i in range(len(dropped)): #iterate over each entry in dropped list
    if len(dropped[i]) > 0: #check if any indecies are present
        print("The following categories where dropped from "+str(dropped[i].name)+"\n")
        for n in range(len(dropped[i])): # iterate over each dropped entry
            print(str(dropped[i][n])) # print each dropped category


In [9]:
## rename categories in categorical variables
for var in categoricalVariables:
    print("The variable "+str(var)+" contains the following catagories")
    for n in range(len(phases_in[var].cat.categories)):
        print(phases_in[var].cat.categories[n])
    print("-------------------------------- \n")
### to be finished
#renameDict = {'dayNight':['Night','Day']}
#phases_in['DayNight'].cat.rename_categories(['Night', 'Day'], inplace=True) # change Daynight from 0/1 to night/Day

The variable Sails contains the following catagories
A1.5-C
A1.5-D
A2-D
J1-B
J1.5-D
J2.5-C
-------------------------------- 

The variable Tack contains the following catagories
Port/Dn
Port/Rch
Port/Up
Stbd/Dn
Stbd/Rch
Stbd/Up
-------------------------------- 

The variable TWS_Bin contains the following catagories
6
8
10
12
14
16
18
20
-------------------------------- 

The variable TWA_Bin contains the following catagories
35
40
45
50
55
120
130
140
150
160
170
-------------------------------- 

The variable BSP_Bin contains the following catagories
6.61
6.69
6.71
6.91
6.93
-------------------------------- 

The variable Headstay contains the following catagories
-12
-8
6
-------------------------------- 

The variable V1 contains the following catagories
-1.0
0.5
-------------------------------- 

The variable D1 contains the following catagories
-3.0
-2.0
0.5
-------------------------------- 

The variable D2 contains the following catagories
-4
-2
0
------------------------------

In [11]:
## create custom bins
phases_in['BspPolBin'] = pd.cut(phases_in['Pol%'], bins=[0, 90, 98, 105, 200], labels=['<90', '90 to 98', '98 to 105', '> 105'])
phases_in['VmgPercBin'] = pd.cut(phases_in['Vmg%'], bins=[0, 85, 95, 105, 200], labels=['<90', '90 to 98', '98 to 105', '> 105'])
phases_in['awaBin'] = pd.cut(phases_in['AWA'], bins=list(range(49,141,2)), labels=list(range(50,140,2)))
phases_in['heelBin'] = pd.cut(abs(phases_in['AHEEL']), bins=list(np.arange(0.5,29.5,1)), labels=list(range(1,29)))
phases_in['BspBin'] = pd.cut(phases_in['BSP'], bins=list(np.arange(3.9,25.1,0.2)), labels=list(np.arange(4,25.1,0.2)))
phases_in['Mode'] = pd.cut(phases_in['TWA'], bins=[0,30,65,120,160,180], labels=['NAHigh','UpWind','Reaching','DownWind','NALow'])
phases_in['TwsBin'] = pd.cut(phases_in['TWS'], bins=list(np.arange(0.5,34.5,1)), labels=list(range(1,34,1)))

#phases_in['TwsBin'] = pd.cut(phases_in['TWS'], bins=list(range(1,34,2)), labels=list(range(2,34,2)))
phases_in['TwaBin'] = pd.cut(phases_in['TWA'], bins=list(range(1,180,2)), labels=list(range(2,180,2)))

In [19]:
## Percentile graphs
# make this easy to select metric and variable
PGVar = 'BspBin'
PGMetric = 'Vmg%'
PGMode = 'UpWind'
filter_remove = {}


PGdataFrame = phases_in[phases_in['Mode']==PGMode]
for key in filter_remove:
    for filter in filter_remove[key]:
        PGdataFrame = PGdataFrame[PGdataFrame['key']!=filter]

PGdataFrame = PGdataFrame[[PGVar,PGMetric,'TwsBin']].dropna()
PGarray = PGdataFrame.to_numpy()


for tws in list(np.unique(PGarray[:,2])):
    PGarray_tws = PGarray[np.where(PGarray==tws)[0],0:2]
    if len(list(np.unique(PGarray_tws[:,0]))) < len(list(PGarray_tws[:,0])):
        percentile_1 = []
        percentile_2 = []
        percentile_3 = []
        percentile_4 = []
        percentile_5 = []
        bins_line = []
        bins_point = []
        points = []
        bins = np.unique(PGarray_tws[:,0])
        for bin in bins:
            sample = PGarray[np.where(PGarray_tws[:,0]==bin),1:2][0]
            if len(sample) > 1:
                bins_line.append(bin)
                score = np.percentile(sample,[50,75,90,95,99])
                percentile_1.append(score[0])
                percentile_2.append(score[1])
                percentile_3.append(score[2])
                percentile_4.append(score[3])
                percentile_5.append(score[4])
            else:
                bins_point.append(bin)
                points.append(sample[0])

        plt.plot(bins_point,points,'bo', alpha=0.5)
        plt.fill_between(
            bins_line,
            percentile_5,
            percentile_4,
            color='blue',
            alpha=0.6)
        plt.fill_between(
            bins_line,
            percentile_4,
            percentile_3,
            color='green',
            alpha=0.75)
        plt.fill_between(
            bins_line,
            percentile_3,
            percentile_2,
            color='orange',
            alpha=0.5)
        plt.fill_between(
            bins_line,
            percentile_2,
            percentile_1,
            color='red',
            alpha=0.3)
        plt.xlim([min(bins_line),max(bins_line)])
        plt.title(str(PGVar)+' Percentile Graph @ TWS = '+str(tws))
        plt.savefig(str(tws)+"_"+str(PGVar)+"PercentileGraph.png") # save plot
        plt.clf() # close plot figure so it is not over plotted

/var/folders/1y/_719kcl11vggrmnzn4bbqdzw0000gn/T/ipykernel_4667/2663627082.py:69: UserWarning: Attempting to set identical left == right == 6.000000000000002 results in singular transformations; automatically expanding.
  plt.xlim([min(bins_line),max(bins_line)])


<Figure size 432x288 with 0 Axes>